In [1]:
import pandas as pd
import numpy as np

In [3]:
payments = pd.read_csv('datasets/payments.csv')
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


The problem with this table is we've order_id repeat more than one time

In [4]:
payments['order_id'].value_counts()

fa65dad1b0e818e3ccc5cb0e39231352    29
ccf804e764ed5650cd8759557269dc13    26
285c2e15bebd4ac83635ccc563dc71f4    22
895ab968e7bb0d5659d16cd74cd1650c    21
fedcd9f7ccdc8cba3a18defedd1a5547    19
                                    ..
6d2a30c9b7dcee3ed507dc9a601f99e7     1
a7737f6d9208dd56ea498a322ed3c37f     1
646e62df54f3e236eb6d5ff3b31429b8     1
e115da7a49ec2acf622e1f31da65cfb9     1
28bbae6599b09d39ca406b747b6632b1     1
Name: order_id, Length: 99440, dtype: int64

This is because each installment was registered with the same order_id

For solve this problem i'll group by order_id and have a table with the total payment and total installments

In [5]:
total_payment = payments.groupby('order_id').agg({'payment_sequential':'max', 'payment_value': 'sum'}).reset_index()
total_payment.head()

,order_id,payment_sequential,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,259.83
2,000229ec398224ef6ca0657da4fc703e,1,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,218.04


This table have information about payment_type,some registers is mixed payment_type

I create a pivot table for transform in columns the payment types

In [9]:
pivot_payment = payments.pivot_table(index='order_id', columns='payment_type', values='payment_value').reset_index()
pivot_payment

payment_type,order_id,boleto,credit_card,debit_card,not_defined,voucher
0,00010242fe8c5a6d1ba2dd792cb16214,NaN,72.19,NaN,NaN,NaN
1,00018f77f2f0320c557190d7a144bdd3,NaN,259.83,NaN,NaN,NaN
2,000229ec398224ef6ca0657da4fc703e,NaN,216.87,NaN,NaN,NaN
3,00024acbcdf0a6daa1e931b038114c75,NaN,25.78,NaN,NaN,NaN
4,00042b26cf59d7ce69dfabb4e55b4fd9,NaN,218.04,NaN,NaN,NaN
...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,343.40,NaN,NaN,NaN,NaN
99436,fffcd46ef2263f404302a634eb57f7eb,386.53,NaN,NaN,NaN,NaN
99437,fffce4705a9662cd70adb13d4a31832d,NaN,116.85,NaN,NaN,NaN
99438,fffe18544ffabc95dfada21779c9644f,NaN,64.71,NaN,NaN,NaN


In [10]:
# Fill NaN values
pivot_payment.fillna(0, inplace=True)

For finish i want to know the payment total and the number of installments, then i'll merge the first and the last table

In [11]:
complete_payment = pivot_payment.merge(total_payment, how='left', on='order_id')
complete_payment


,order_id,boleto,credit_card,debit_card,not_defined,voucher,payment_sequential,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,0.00,72.19,0.0,0.0,0.0,1,72.19
1,00018f77f2f0320c557190d7a144bdd3,0.00,259.83,0.0,0.0,0.0,1,259.83
2,000229ec398224ef6ca0657da4fc703e,0.00,216.87,0.0,0.0,0.0,1,216.87
3,00024acbcdf0a6daa1e931b038114c75,0.00,25.78,0.0,0.0,0.0,1,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,0.00,218.04,0.0,0.0,0.0,1,218.04
...,...,...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,343.40,0.00,0.0,0.0,0.0,1,343.40
99436,fffcd46ef2263f404302a634eb57f7eb,386.53,0.00,0.0,0.0,0.0,1,386.53
99437,fffce4705a9662cd70adb13d4a31832d,0.00,116.85,0.0,0.0,0.0,1,116.85
99438,fffe18544ffabc95dfada21779c9644f,0.00,64.71,0.0,0.0,0.0,1,64.71


In [12]:
complete_payment.columns

Index(['order_id', 'boleto', 'credit_card', 'debit_card', 'not_defined',
       'voucher', 'payment_sequential', 'payment_value'],
      dtype='object')

In [16]:
complete_payment.to_csv("datasets/processed/payments.csv", index=False)